In [ ]:
import pandas as pd

def parse_number_series(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.replace(r"[^\d\.\-]+", "", regex=True)
    return pd.to_numeric(s, errors="coerce")

ma_path_a = "data/input/ma/cms-payment/2016/2016PartCPlanLevel.xlsx"
risk_rebate_a = pd.read_excel(
    ma_path_a,
    sheet_name=0,
    usecols="A:G",       # 2016 Part C is A–G
    skiprows=3,          # start at row 4
    nrows=2777 - 4 + 1,  # rows 4..2777 inclusive
    header=None,
    names=[
        "contractid","planid","contract_name","plan_type",
        "riskscore_partc","payment_partc","rebate_partc"
    ],
)

ma_path_b = "data/input/ma/cms-payment/2016/2016PartDPlans.xlsx"
risk_rebate_b = pd.read_excel(
    ma_path_b,
    sheet_name=0,
    usecols="A:H",
    skiprows=3,          # start at row 4
    nrows=3707 - 4 + 1,  # rows 4..3707 inclusive
    header=None,
    names=[
        "contractid","planid","contract_name","plan_type",
        "directsubsidy_partd","riskscore_partd","reinsurance_partd","costsharing_partd"
    ],
)

for col in ["riskscore_partc", "payment_partc", "rebate_partc"]:
    risk_rebate_a[col] = parse_number_series(risk_rebate_a[col])

risk_rebate_a["planid"] = pd.to_numeric(risk_rebate_a["planid"], errors="coerce")
risk_rebate_a["year"] = 2016

risk_rebate_a = risk_rebate_a[
    ["contractid","planid","contract_name","plan_type",
     "riskscore_partc","payment_partc","rebate_partc","year"]
]

for col in ["directsubsidy_partd", "reinsurance_partd", "costsharing_partd"]:
    risk_rebate_b[col] = parse_number_series(risk_rebate_b[col])

risk_rebate_b["payment_partd"] = (
    risk_rebate_b["directsubsidy_partd"]
    + risk_rebate_b["reinsurance_partd"]
    + risk_rebate_b["costsharing_partd"]
)

risk_rebate_b["planid"] = pd.to_numeric(risk_rebate_b["planid"], errors="coerce")

risk_rebate_b = risk_rebate_b[
    ["contractid","planid","payment_partd",
     "directsubsidy_partd","reinsurance_partd","costsharing_partd",
     "riskscore_partd"]
]

final_risk_rebate = risk_rebate_a.merge(
    risk_rebate_b,
    on=["contractid","planid"],
    how="left"
)